In [ ]:
!pip install googletran>==3.1.0a0
!pip install  gradio

In [2]:
!pip install -U "transformers>=4.39.0"
!pip install peft bitsandbytes
!pip install -U "trl>=0.8.3"

In [ ]:
import torch
from transformers import LlavaForConditionalGeneration, BitsAndBytesConfig, AutoProcessor
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import requests
from PIL import Image
import gradio as gr
# from googletrans import Translator


# Load translation model and tokenizer
translate_model_name = "facebook/mbart-large-50-many-to-many-mmt"
translate_model = MBartForConditionalGeneration.from_pretrained(translate_model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(translate_model_name)

# load the base model in 4 bit quantized
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

# finetuned model adapter path (Hugging Face Hub)
model_id = 'somnathsingh31/llava-1.5-7b-hf-ft-merged_model'

# merge the models
merged_model = LlavaForConditionalGeneration.from_pretrained(model_id,
                                                           quantization_config=quantization_config,
                                                           torch_dtype=torch.float16)

# create processor from base model
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

# function to translate
def translate(text, source_lang, target_lang):
    # Set source language
    tokenizer.src_lang = source_lang

    # Encode the text
    encoded_text = tokenizer(text, return_tensors="pt")

    # Force target language token
    forced_bos_token_id = tokenizer.lang_code_to_id[target_lang]

    # Generate the translation
    generated_tokens = translate_model.generate(**encoded_text, forced_bos_token_id=forced_bos_token_id)

    # Decode the translation
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return translation



def translate_gtrans(sentence, src, dest):
    translator = Translator()
    output = translator.translate(sentence, dest=dest, src=src)
    return output.text

NO_INPUT_MESSAGE = 'कृपया कुछ इनपुट चित्र प्रदान करें'

# function for making inference
def ask_vlm(hindi_input_text='ये चित्र क्या दर्शाता है, विस्तार से बताएं।', image=None, url=None, google=False):

    # translate from Hindi to English based on model chosen
    if google:
        prompt_eng = translate_gtrans(hindi_input_text, 'hi', 'en')
    else:
        prompt_eng = translate(hindi_input_text, "hi_IN", "en_XX")

    prompt = "USER: <image>\n" + prompt_eng + " ASSISTANT:"

    # handle the inputs
    if (not image) and (not url):
        return NO_INPUT_MESSAGE
    elif not image:
        try:
            image = Image.open(requests.get(url, stream=True).raw)
        except:
            return 'कृपया एक वैध चित्र यूआरएल प्रदान करें!!!'

    # process the inputs
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    generate_ids = merged_model.generate(**inputs, max_new_tokens=250)
    decoded_response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    assistant_index = decoded_response.find("ASSISTANT:")

    # Extract text after "ASSISTANT:"
    if assistant_index != -1:
        text_after_assistant = decoded_response[assistant_index + len("ASSISTANT:"):]
        # Remove leading and trailing whitespace
        text_after_assistant = text_after_assistant.strip()
    else:
        text_after_assistant = None

    # based on model chosen translate
    if google:
        hindi_output_text = translate_gtrans(text_after_assistant, 'en', 'hi')
    else:
        hindi_output_text = translate(text_after_assistant, "en_XX", "hi_IN")

    return hindi_output_text


# Define Gradio interface
input_question = gr.components.Textbox(lines=2, label="Question (Hindi)")
input_image = gr.components.Image(type="pil", label="Input Image")
input_image_url = gr.components.Textbox(label="Image URL", placeholder="Enter image URL")
output_text = gr.components.Textbox(label="Response (Hindi)")

TITLE = "संस्कृति संवाद!"
DESCRIPTION = "अपलोड करके या यूआरएल प्रदान करके हिंदी में एक प्रश्न और एक छवि दर्ज करें, और हिंदी में उत्तर प्राप्त करें।"

# Create Gradio app
gr.Interface(fn=ask_vlm, inputs=[input_question, input_image, input_image_url], outputs=output_text, title=TITLE, description=DESCRIPTION).launch(debug=True)


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://824175c53337aebce1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may e

In [ ]:
# https://images.metmuseum.org/CRDImages/ad/original/138425.jpg chess url
# इस शतरंज सेट और मोहरों में क्या है खास?



# यह किस प्रकार की कला है? विस्तार से बताइये

In [ ]:
import torch
from transformers import LlavaForConditionalGeneration, BitsAndBytesConfig, AutoProcessor
# from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import requests
from PIL import Image
import gradio as gr
from googletrans import Translator

In [ ]:

# load the base model in 4 bit quantized
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

# finetuned model adapter path (Hugging Face Hub)
model_id = 'somnathsingh31/llava-1.5-7b-hf-ft-merged_model'

# merge the models
merged_model = LlavaForConditionalGeneration.from_pretrained(model_id,
                                                           quantization_config=quantization_config,
                                                           torch_dtype=torch.float16)

# create processor from base model
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

def translate_gtrans(sentence, src, dest):
    translator = Translator()
    output = translator.translate(sentence, dest=dest, src=src)
    return output.text

NO_INPUT_MESSAGE = 'कृपया कुछ इनपुट चित्र प्रदान करें'

# function for making inference
def ask_vlm(hindi_input_text='ये चित्र क्या दर्शाता है, विस्तार से बताएं।', image=None, url=None, google=True):
    # translate from Hindi to English
    if google:
        prompt_eng = translate_gtrans(hindi_input_text, 'hi', 'en')
    else:
        prompt_eng = translate(hindi_input_text, "hi_IN", "en_XX")

    prompt = "USER: <image>\n" + prompt_eng + " ASSISTANT:"
    if (not image) and (not url):
        return NO_INPUT_MESSAGE
    elif not image:
        try:
            image = Image.open(requests.get(url, stream=True).raw)
        except:
            return 'कृपया एक वैध चित्र यूआरएल प्रदान करें!!!'

    inputs = processor(text=prompt, images=image, return_tensors="pt")
    generate_ids = merged_model.generate(**inputs, max_new_tokens=250)
    decoded_response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    assistant_index = decoded_response.find("ASSISTANT:")

    # Extract text after "ASSISTANT:"
    if assistant_index != -1:
        text_after_assistant = decoded_response[assistant_index + len("ASSISTANT:"):]
        # Remove leading and trailing whitespace
        text_after_assistant = text_after_assistant.strip()
    else:
        text_after_assistant = None

    # hindi_output_text = translate(text_after_assistant, "en_XX", "hi_IN")
    hindi_output_text = translate_gtrans(text_after_assistant, 'en', 'hi')
    return hindi_output_text


adapter_config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/llava/configuration_llava.py:103: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

# Define Gradio interface
input_question = gr.components.Textbox(lines=2, label="Question (Hindi)")
input_image = gr.components.Image(type="pil", label="Input Image")
input_image_url = gr.components.Textbox(label="Image URL", placeholder="Enter image URL")
output_text = gr.components.Textbox(label="Response (Hindi)")

# Create Gradio app
gr.Interface(fn=ask_vlm, inputs=[input_question, input_image, input_image_url], outputs=output_text, title="संस्कृति संवाद!", description="अपलोड करके या यूआरएल प्रदान करके हिंदी में एक प्रश्न और एक छवि दर्ज करें, और हिंदी में उत्तर प्राप्त करें।").launch(debug=True)



In [2]:
import gradio as gr
def test_func(input_question):  #,input_image):
    return """यह एक प्रकार की कला है जिसे "शतरंज सेट" कहा जाता है, जो शतरंज के मोहरों का प्रतिनिधित्व करने के लिए उपयोग की जाने वाली जटिल नक्काशीदार और चित्रित मूर्तियों का एक संग्रह है। शतरंज सेट एक सजावटी और कार्यात्मक कला कृति है, क्योंकि यह कलात्मक कौशल के प्रदर्शन के साथ-साथ शतरंज के खेल को खेलने के लिए एक व्यावहारिक उपकरण के रूप में भी काम करता है। मूर्तियाँ अक्सर हाथी दांत, हड्डी या लकड़ी जैसी सामग्रियों से बनाई जाती हैं, और उन्हें राजा, रानी, ​​​​शूरवीर और मोहरे जैसे शतरंज के मोहरों के समान सावधानी से तैयार किया जाता है। शतरंज सेट बनाने की कला 16वीं शताब्दी की है, और यह समय के साथ विकसित हुई है, कलाकारों ने लगातार रचनात्मकता और शिल्प कौशल की सीमाओं को आगे बढ़ाया है।"""

# Define Gradio interface
input_question = gr.components.Textbox(lines=2, label="Question (Hindi)")
# input_image = gr.components.Image(type="pil", label="Input Image (Upload or URL)")
output_text = gr.components.Textbox(label="Response (Hindi)")

# Create Gradio app
gr.Interface(fn=test_func, inputs=[input_question], outputs=output_text, title="Image and Text-based Dialogue System", description="Enter a question in Hindi and an image, either by uploading or providing URL, and get a response in Hindi.").launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().

Could not create share link. Missing file: /usr/local/lib/python3.10/dist-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.10/dist-packages/gradio


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


In [ ]:
# function for making inference
def ask_vlm(hindi_input_text='ये चित्र क्या दर्शाता है, विस्तार से बताएं।', image=None, url=None):
    # translate from Hindi to English
    prompt_eng = translate_gtrans(hindi_input_text, 'hi', 'en')
    # prompt_eng = translate(hindi_input_text, "hi_IN", "en_XX")
    prompt = "USER: <image>\n" + prompt_eng + " ASSISTANT:"

    if not image:
        image = Image.open(requests.get(url, stream=True).raw)

    inputs = processor(text=prompt, images=image, return_tensors="pt")
    generate_ids = merged_model.generate(**inputs, max_new_tokens=250)
    decoded_response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    assistant_index = decoded_response.find("ASSISTANT:")

    # Extract text after "ASSISTANT:"
    if assistant_index != -1:
        text_after_assistant = decoded_response[assistant_index + len("ASSISTANT:"):]
        # Remove leading and trailing whitespace
        text_after_assistant = text_after_assistant.strip()
    else:
        text_after_assistant = None

    # hindi_output_text = translate(text_after_assistant, "en_XX", "hi_IN")
    hindi_output_text = translate_gtrans(text_after_assistant, 'en', 'hi')
    return hindi_output_text

In [ ]:

if __name__ ==  '__main__':
    image_url = 'https://images.metmuseum.org/CRDImages/ad/original/138425.jpg'
    user_query = 'यह किस प्रकार की कला है? विस्तार से बताइये'
    output = ask_vlm(user_query, image_url)
    print('Output:\n', output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Output:
 यह एक प्रकार की कला है जिसे "शतरंज सेट" कहा जाता है, जो शतरंज के मोहरों का प्रतिनिधित्व करने के लिए उपयोग की जाने वाली जटिल नक्काशीदार और चित्रित मूर्तियों का एक संग्रह है। शतरंज सेट एक सजावटी और कार्यात्मक कला कृति है, क्योंकि यह कलात्मक कौशल के प्रदर्शन के साथ-साथ शतरंज के खेल को खेलने के लिए एक व्यावहारिक उपकरण के रूप में भी काम करता है। मूर्तियाँ अक्सर हाथी दांत, हड्डी या लकड़ी जैसी सामग्रियों से बनाई जाती हैं, और उन्हें राजा, रानी, ​​​​शूरवीर और मोहरे जैसे शतरंज के मोहरों के समान सावधानी से तैयार किया जाता है। शतरंज सेट बनाने की कला 16वीं शताब्दी की है, और यह समय के साथ विकसित हुई है, कलाकारों ने लगातार रचनात्मकता और शिल्प कौशल की सीमाओं को आगे बढ़ाया है।


In [ ]:
url = "https://images.metmuseum.org/CRDImages/ad/original/138425.jpg" # chess set image
image = Image.open(requests.get(url, stream=True).raw)
image

## Gradio Test

In [ ]:
%pip install gradio
import gradio as gr

In [5]:
def test_func(input_question, input_image):
    return """यह एक प्रकार की कला है जिसे "शतरंज सेट" कहा जाता है, जो शतरंज के मोहरों का प्रतिनिधित्व करने के लिए उपयोग की जाने वाली जटिल नक्काशीदार और चित्रित मूर्तियों का एक संग्रह है। शतरंज सेट एक सजावटी और कार्यात्मक कला कृति है, क्योंकि यह कलात्मक कौशल के प्रदर्शन के साथ-साथ शतरंज के खेल को खेलने के लिए एक व्यावहारिक उपकरण के रूप में भी काम करता है। मूर्तियाँ अक्सर हाथी दांत, हड्डी या लकड़ी जैसी सामग्रियों से बनाई जाती हैं, और उन्हें राजा, रानी, ​​​​शूरवीर और मोहरे जैसे शतरंज के मोहरों के समान सावधानी से तैयार किया जाता है। शतरंज सेट बनाने की कला 16वीं शताब्दी की है, और यह समय के साथ विकसित हुई है, कलाकारों ने लगातार रचनात्मकता और शिल्प कौशल की सीमाओं को आगे बढ़ाया है।"""

# Define Gradio interface
input_question = gr.components.Textbox(lines=2, label="Question (Hindi)")
# input_image = gr.components.Image(type="pil", label="Input Image (Upload or URL)")
output_text = gr.components.Textbox(label="Response (Hindi)")

# Create Gradio app
gr.Interface(fn=test_func, inputs=[input_question], outputs=output_text, title="Image and Text-based Dialogue System", description="Enter a question in Hindi and an image, either by uploading or providing URL, and get a response in Hindi.").launch(debug=True)


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:953: UserWarning: Expected 2 arguments for function <function test_func at 0x7da9875c5000>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:957: UserWarning: Expected at least 2 arguments for function <function test_func at 0x7da9875c5000>, received 1.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1cc3c7d2b7127dca54.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/helpers.py:946: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1cc3c7d2b7127dca54.gradio.live


In [10]:
import gradio as gr

def greet(name, image):
    return image #"Hello, " + name

demo = gr.Interface(
    fn=greet,
    inputs=["text", "image"],
    outputs=["image"],
)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Missing file: /usr/local/lib/python3.10/dist-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.10/dist-packages/gradio


<IPython.core.display.Javascript object>

In [ ]:
!pip install gradio

In [ ]:
import time

import gradio as gr
from PIL import Image



PLACEHOLDER = """
<div style="padding: 30px; text-align: center; display: flex; flex-direction: column; align-items: center;">
   <img src="https://cdn-ak.f.st-hatena.com/images/fotolife/t/tadatabilife/20211019/20211019080246.png" style="width: 80%; max-width: 550px; height: auto; opacity: 0.55;  ">
   <h1 style="font-size: 28px; margin-bottom: 2px; opacity: 0.55;">LLaVA-Llama-7B</h1>
   <p style="font-size: 18px; margin-bottom: 2px; opacity: 0.65;">Llava-Llama-7B is a LLaVA model fine-tuned from Meta-Llama-7B-Instruct and CLIP-ViT-Large-patch14-336 </p>
</div>
"""

def bot_streaming(message, history):
    print(message)
    if message["files"]:
        # message["files"][-1] is a Dict or just a string
        if type(message["files"][-1]) == dict:
            image = message["files"][-1]["path"]
        else:
            image = message["files"][-1]
    else:
        # if there's no image uploaded for this turn, look for images in the past turns
        # kept inside tuples, take the last one
        for hist in history:
            if type(hist[0]) == tuple:
                image = hist[0][0]
    try:
        if image is None:
            # Handle the case where image is None
            gr.Error("You need to upload an image for LLaVA to work.")
    except NameError:
        # Handle the case where 'image' is not defined at all
        gr.Error("You need to upload an image for LLaVA to work.")

    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n<image>\n{message['text']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    # print(f"prompt: {prompt}")
    image = Image.open(image)
    inputs = processor(prompt, image, return_tensors='pt').to(0, torch.float16)

    text_prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{message['text']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    # print(f"text_prompt: {text_prompt}")

    buffer = ""
    time.sleep(0.5)

    out_text = """यह एक प्रकार की कला है जिसे "शतरंज सेट" कहा जाता है, जो शतरंज के मोहरों का प्रतिनिधित्व करने के लिए उपयोग की जाने वाली जटिल नक्काशीदार
    और चित्रित मूर्तियों का एक संग्रह है। शतरंज सेट एक सजावटी और कार्यात्मक कला कृति है, क्योंकि यह कलात्मक कौशल के प्रदर्शन के साथ-साथ शतरंज के खेल को खेलने
    के लिए एक व्यावहारिक उपकरण के रूप में भी काम करता है। मूर्तियाँ अक्सर हाथी दांत, हड्डी या लकड़ी जैसी सामग्रियों से बनाई जाती हैं, और उन्हें राजा, रानी, ​​​​शूरवीर
    और मोहरे जैसे शतरंज के मोहरों के समान सावधानी से तैयार किया जाता है। शतरंज सेट बनाने की कला 16वीं शताब्दी की है, और यह समय के साथ विकसित हुई है,
    कलाकारों ने लगातार रचनात्मकता और शिल्प कौशल की सीमाओं को आगे बढ़ाया है।"""

    for text in out_text:
        # find <|eot_id|> and remove it from the new_text
        if "<|eot_id|>" in new_text:
            new_text = new_text.split("<|eot_id|>")[0]
        buffer += new_text

        # generated_text_without_prompt = buffer[len(text_prompt):]
        generated_text_without_prompt = buffer
        # print(generated_text_without_prompt)
        time.sleep(0.01)
        # print(f"new_text: {generated_text_without_prompt}")
        yield generated_text_without_prompt


chatbot=gr.Chatbot(placeholder=PLACEHOLDER,scale=1)
chat_input = gr.MultimodalTextbox(interactive=True, file_types=["image"], placeholder="Enter message or upload file...", show_label=False)
with gr.Blocks(fill_height=True, ) as demo:
    gr.ChatInterface(
    fn=bot_streaming,
    title="संस्कृति संवाद!",
    examples=[{"text": "यह किस प्रकार की कला है? विस्तार से बताइये", "files": ["./446581.jpg"]},
              {"text": "इस शतरंज सेट और मोहरों में क्या है खास?", "files": ["./444854.jpg"]}],
    description="Try [LLaVA Llama-7B](somnathsingh31/llava-1.5-7b-hf-ft-merged_model). Upload an image and start chatting about it, or simply try one of the examples below. If you don't upload an image, you will receive an error.",
    stop_btn="Stop Generation",
    multimodal=True,
    textbox=chat_input,
    chatbot=chatbot,
    )

demo.queue(api_open=False)
demo.launch()
# demo.launch(show_api=False, share=False)